In [1]:
import datetime as dt
import json
import scipy as sp
from tqdm import tqdm
from functions import bfs, find_closest_nodes, find_route, get_distance, get_way
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

far route


In [2]:
boxes = ['dwtest20', 'dwtest24', 'dwtest23', 'dwtest17', 'dwtest10', 'dwtest23', 'dwtest19', 
         'dwtest29', 'dwtest15', 'dwtest28', 'dwtest26']

In [3]:
raw_nov = pd.read_csv('raw_data11.csv', sep=',', decimal=',')
raw_dec = pd.read_csv('raw_data12.csv', sep=',', decimal=',')
raw_jan = pd.read_csv('raw_data01.csv', sep=',', decimal=',')
raw_feb = pd.read_csv('raw_data02.csv', sep=',', decimal=',')
raw_mar = pd.read_csv('raw_data03.csv', sep=',', decimal=',')

In [74]:
total = len(raw_mar)
empty = len(raw_mar[raw_mar.accel_forward.isna()])
rel = empty / total 
print(rel)

0.786131544785807


In [4]:
november = raw_nov.sort_values(by='packtime').drop_duplicates(subset='packtime').reset_index(drop=True)
december = raw_dec.sort_values(by='packtime').drop_duplicates(subset='packtime').reset_index(drop=True)
january = raw_jan.sort_values(by='packtime').drop_duplicates(subset='packtime').reset_index(drop=True)
february = raw_feb.sort_values(by='packtime').drop_duplicates(subset='packtime').reset_index(drop=True)

In [38]:
dataset1 = pd.concat([november, december])

In [29]:
dataset1 = dataset.sort_values(by='packtime').reset_index(drop=True)

In [39]:
dataset2 = pd.concat([january, february])

In [45]:
def set_hit(data):

    proc = []

    for dw in data.device_id.unique():

        if dw in boxes:

            box = data[data.device_id == dw].reset_index(drop=True)
            box['unixtime'] = 0

            for i, row in box.iterrows():

                box.loc[i, 'unixtime'] = dt.datetime.timestamp(pd.to_datetime(row.packtime))

            box['forward'] = box.unixtime - box.unixtime.shift(1)
            box['maybe_hit'] = ( (box.forward < 7100) | ((box.forward > 7300) & (box.forward < 14300)) | 
                                ((box.forward > 14500) & (box.forward < 21500)) | (box.forward > 21700) )
            proc.append(box)
    
    hits = pd.concat(proc)
    
    hits['len_accels'] = 0
    hits['probably_hit'] = False
    hits = hits.reset_index(drop=True)
    for j, line in hits.iterrows():
        
        try:
            lf = len(line.accel_forward.strip('{}').split(sep=',')) 
            lv = len(line.accel_vertical.strip('{}').split(sep=',')) 
            lt = len(line.accel_transverse.strip('{}').split(sep=','))
            
        except:
            
            continue
        hits.loc[j, 'len_accels'] = lf
        hits.loc[j, 'probably_hit'] = ( hits.loc[j, 'len_accels'] < 600 )
    
    
    return hits

In [46]:
hits_b4ny = set_hit(dataset1)
hits_afterny = set_hit(dataset2)

In [68]:
hits_afterny.tail(50)

,record_id,device_id,pack_id,rwcarfw,mestypeid,packtime,latitude,longitude,velocity,direction,temperature,rssi,batmain,area_id,g_range,g_freq,g_time,accel_forward,accel_vertical,accel_transverse,imsi,batgw1,tcargo,pcargo,vcargo,getdata,odometer,received_comup,unixtime,forward,maybe_hit,len_accels,probably_hit
17207,245872,dwtest20,1850,41714,0,2024-02-25 16:15:42.000000,0.000000,0.000000,0.0,0.000,16,80,10087,NaN,4,200,3,NaN,NaN,NaN,250510200025677,0,0,0,0,2024-02-25 16:15:46.848475,3004780,False,1708866942,14582.0,True,0,False
17208,245916,dwtest20,1851,41714,0,2024-02-25 18:17:11.000000,0.000000,0.000000,0.0,0.000,16,82,10065,NaN,4,200,3,NaN,NaN,NaN,250510200025677,0,0,0,0,2024-02-25 18:17:15.219208,3004780,False,1708874231,7289.0,False,0,False
17209,245958,dwtest20,1852,41714,0,2024-02-25 20:18:41.000000,0.000000,0.000000,0.0,0.000,16,84,10120,NaN,4,200,3,NaN,NaN,NaN,250510200025677,0,0,0,0,2024-02-25 20:18:49.946388,3004780,False,1708881521,7290.0,False,0,False
17210,246001,dwtest20,1853,41714,0,2024-02-25 22:20:15.000000,0.000000,0.000000,0.0,0.000,16,72,10131,NaN,4,200,3,NaN,NaN,NaN,250510200025677,0,0,0,0,2024-02-25 22:20:21.969708,3004780,False,1708888815,7294.0,False,0,False
17211,246049,dwtest20,1854,41714,0,2024-02-26 00:22:44.000000,0.000000,0.000000,0.0,0.000,17,76,10131,NaN,4,200,3,NaN,NaN,NaN,250510200025677,0,0,0,0,2024-02-26 00:22:47.813593,3004780,False,1708896164,7349.0,True,0,False
17212,246092,dwtest20,1855,41714,0,2024-02-26 02:24:11.000000,0.000000,0.000000,0.0,0.000,16,84,10131,NaN,4,200,3,NaN,NaN,NaN,250510200025677,0,0,0,0,2024-02-26 02:24:17.748949,3004780,False,1708903451,7287.0,False,0,False
17213,246140,dwtest20,1856,41714,0,2024-02-26 04:25:40.000000,0.000000,0.000000,0.0,0.000,17,72,10120,NaN,4,200,3,NaN,NaN,NaN,250510200025677,0,0,0,0,2024-02-26 04:25:45.789330,3004780,False,1708910740,7289.0,False,0,False
17214,246185,dwtest20,1857,41714,0,2024-02-26 06:27:07.000000,0.000000,0.000000,0.0,0.000,17,82,10120,NaN,4,200,3,NaN,NaN,NaN,250510200025677,0,0,0,0,2024-02-26 06:27:11.630174,3004780,False,1708918027,7287.0,False,0,False
17215,246230,dwtest20,1858,41714,0,2024-02-26 08:28:38.000000,0.000000,0.000000,0.0,0.000,17,84,10120,NaN,4,200,3,NaN,NaN,NaN,250510200025677,0,0,0,0,2024-02-26 08:28:45.814500,3004780,False,1708925318,7291.0,False,0,False
17216,246277,dwtest20,1859,41714,0,2024-02-26 10:30:28.000000,0.000000,0.000000,0.0,0.000,17,68,10131,NaN,4,200,3,NaN,NaN,NaN,250510200025677,0,0,0,0,2024-02-26 10:30:34.210847,3004780,False,1708932628,7310.0,True,0,False


In [70]:
len(hits_afterny[hits_afterny.accel_forward.isna()])

13022

In [23]:
hits_b4ny

,record_id,device_id,pack_id,rwcarfw,mestypeid,packtime,latitude,longitude,velocity,direction,temperature,rssi,batmain,area_id,g_range,g_freq,g_time,accel_forward,accel_vertical,accel_transverse,imsi,batgw1,tcargo,pcargo,vcargo,getdata,odometer,received_comup,unixtime,forward,maybe_hit,len_accels,probably_hit
0,117328,dwtest24,4626,41616,0,2023-11-01 00:03:31.000000,67.144504,63.439013,53.7,81.500,-13,88,9086,NaN,16,200,3,"{-1152,-816,-960,-960,-1008,-1008,-1056,-1152,...","{-48,48,96,-144,48,-240,96,192,48,0,-96,-192,-...","{0,-96,-48,96,0,-144,48,96,-48,-48,0,48,96,96,...",2.505102e+14,0,0,0,0,2023-11-01 10:05:14.079802,0,False,1698786211,NaN,False,600,False
1,117329,dwtest24,4627,41616,0,2023-11-01 02:03:45.000000,67.371834,63.920064,0.0,230.350,-14,82,9295,NaN,16,200,3,"{-1008,-1008,-1008,-1008,-1008,-1008,-960,-960...","{0,0,-48,-48,-48,-48,0,-48,0,0,0,-48,0,-48,-48...","{0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,48,48,0,0,0...",2.505102e+14,0,0,0,0,2023-11-01 10:05:35.090317,0,False,1698793425,7214.0,False,600,False
2,117111,dwtest24,4628,41616,0,2023-11-01 04:03:13.000000,67.470714,64.028742,0.3,237.370,-14,62,9207,NaN,16,200,3,"{-960,-1008,-1008,-1008,-960,-960,-1008,-960,-...","{-48,0,0,0,0,0,0,0,0,0,0,-48,0,0,0,0,0,0,0,0,0...","{0,48,0,0,0,0,0,0,0,48,48,0,0,0,0,0,0,0,0,0,0,...",2.505102e+14,0,0,0,0,2023-11-01 04:03:33.027478,0,False,1698800593,7168.0,False,600,False
3,117181,dwtest24,4629,41616,0,2023-11-01 06:03:13.000000,67.470694,64.028667,0.1,65.100,-13,52,9207,NaN,16,200,3,"{-960,-1008,-960,-1008,-960,-1008,-960,-960,-9...","{0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-48,-48,0,0,0,-...","{0,0,0,0,0,0,0,0,0,0,0,0,0,48,0,0,0,0,0,0,0,0,...",2.505102e+14,0,0,0,0,2023-11-01 06:03:30.961324,0,False,1698807793,7200.0,False,600,False
4,117245,dwtest24,4630,41616,0,2023-11-01 08:03:16.000000,67.470944,64.029089,0.8,116.680,-11,52,9262,NaN,16,200,3,"{-960,-960,-960,-960,-1008,-960,-1008,-1008,-1...","{-48,-48,0,0,0,-48,0,0,-48,0,0,0,0,0,0,-48,0,0...","{0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",2.505102e+14,0,0,0,0,2023-11-01 08:03:33.689675,0,False,1698814996,7203.0,False,600,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6865,174986,dwtest20,356,41616,0,2023-12-25 20:54:37.000000,65.990269,60.319197,0.7,350.370,-8,54,6655,NaN,16,200,3,"{-960,-912,-960,-912,-960,-960,-960,-960,-960,...","{-48,-48,-48,-96,-48,-96,-96,-96,-96,-48,-96,-...","{0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",2.505102e+14,0,0,0,0,2023-12-25 20:54:57.119256,0,False,1703526877,7198.0,False,600,False
6866,175065,dwtest20,357,41616,0,2023-12-25 22:54:37.000000,65.990816,60.320631,0.2,132.480,-8,52,6413,NaN,16,200,3,"{-912,-912,-960,-960,-960,-960,-960,-960,-912,...","{-48,-48,-48,-96,-48,-48,-48,-48,-48,-96,-96,-...","{0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",2.505102e+14,0,0,0,0,2023-12-25 22:55:17.366591,0,False,1703534077,7200.0,False,600,False
6867,175137,dwtest20,358,41616,0,2023-12-26 00:54:42.000000,65.990738,60.320521,0.4,33.950,-8,54,5764,NaN,16,200,3,"{-960,-960,-960,-960,-960,-912,-960,-960,-960,...","{-48,-48,-48,-48,-48,-96,-96,-96,-96,-96,-96,-...","{0,48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2.505102e+14,0,0,0,0,2023-12-26 00:55:01.890182,0,False,1703541282,7205.0,False,600,False
6868,175702,dwtest20,377,41616,1,2023-12-26 18:50:38.000000,65.158720,57.233408,0.9,161.590,-7,64,5764,NaN,16,200,3,"{-1056,-912,-864,-960,-1104,-1008,-912,-336,91...","{-48,-48,0,0,-48,-48,0,336,480,1296,1248,1344,...","{0,0,0,0,0,-48,0,-96,48,432,-672,240,0,-192,-8...",2.505102e+14,0,0,0,0,2023-12-26 18:50:44.320633,0,False,1703605838,64556.0,True,32,True


In [10]:
nov_hits = set_hit(november)
dec_hits = set_hit(december)
jan_hits = set_hit(january)
feb_hits = set_hit(february)

In [ ]:
nov_hits.to_excel('nov_hits.xlsx')
dec_hits.to_excel('dec_hits.xlsx')
jan_hits.to_excel('jan_hits.xlsx')
feb_hits.to_excel('feb_hits.xlsx')

In [32]:
total = len(hits_b4ny)
badtimes = len(hits_b4ny[hits_b4ny.maybe_hit])
share_badtimes = (badtimes / total)
calchits = len(hits_b4ny[hits_b4ny.mestypeid == 1])
share_calchits = calchits / total
accelhits = len(hits_b4ny[hits_b4ny.probably_hit])
share_accelhits = accelhits / total

corel_calchits_badtimes = len(hits_b4ny[hits_b4ny.mestypeid == hits_b4ny.maybe_hit]) / total
corel_calchits_accelhits = len(hits_b4ny[hits_b4ny.mestypeid == hits_b4ny.probably_hit]) / total
corel_badtimes_accelhits = len(hits_b4ny[hits_b4ny.maybe_hit == hits_b4ny.probably_hit]) / total

print(f' всего - {total}\n не 2 часа - {badtimes}, доля не 2 часа - {share_badtimes} \n по местайп - {calchits}, доля по местайп - {share_calchits}\n по акселерометру - {accelhits}, доля по акселерометру - {share_accelhits} \n {corel_calchits_badtimes} {corel_calchits_accelhits} {corel_badtimes_accelhits}')

 всего - 6870
 не 2 часа - 645, доля не 2 часа - 0.09388646288209607 
 по местайп - 594, доля по местайп - 0.08646288209606987
 по акселерометру - 594, доля по акселерометру - 0.08646288209606987 
 0.991703056768559 1.0 0.991703056768559


In [47]:
total = len(hits_afterny)
badtimes = len(hits_afterny[hits_afterny.maybe_hit])
share_badtimes = (badtimes / total)
calchits = len(hits_afterny[hits_afterny.mestypeid == 1])
share_calchits = calchits / total
accelhits = len(hits_afterny[hits_afterny.probably_hit])
share_accelhits = accelhits / total

corel_calchits_badtimes = len(hits_afterny[hits_afterny.mestypeid == hits_afterny.maybe_hit]) / total
corel_calchits_accelhits = len(hits_afterny[hits_afterny.mestypeid == hits_afterny.probably_hit]) / total
corel_badtimes_accelhits = len(hits_afterny[hits_afterny.maybe_hit == hits_afterny.probably_hit]) / total

print(f' всего - {total}\n не 2 часа - {badtimes}, доля не 2 часа - {share_badtimes} \n по местайп - {calchits}, доля по местайп - {share_calchits}\n по акселерометру - {accelhits}, доля по акселерометру - {share_accelhits} \n {corel_calchits_badtimes} {corel_calchits_accelhits} {corel_badtimes_accelhits}')

 всего - 17257
 не 2 часа - 11838, доля не 2 часа - 0.6859824998551313 
 по местайп - 599, доля по местайп - 0.03471055223967086
 по акселерометру - 579, доля по акселерометру - 0.03355160224836298 
 0.34872805238453963 0.9879469200903981 0.34756910239323174


In [49]:
hits_afterny.to_excel('hits_after_ny.xlsx')
hits_b4ny.to_excel('hits_before_ny.xlsx')